In [1]:
# Step 1: Import Libraries
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys

In [2]:
# Step 2: Set Up Kaggle Environment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
# Step 3: Load and Preprocess the Dataset
dataset_path = '/kaggle/input/dogs-vs-cats/Exp3Dataset/kagglecatsanddogs_3367a/PetImages'
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = datasets.ImageFolder(root=dataset_path, transform=data_transforms)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

In [4]:
# Step 4: Define the CNN Model
class CNN(nn.Module):
    def __init__(self, activation_func, weight_init):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 2)
        
        if weight_init == 'xavier':
            nn.init.xavier_uniform_(self.conv1.weight)
            nn.init.xavier_uniform_(self.conv2.weight)
            nn.init.xavier_uniform_(self.fc1.weight)
            nn.init.xavier_uniform_(self.fc2.weight)
        elif weight_init == 'kaiming':
            nn.init.kaiming_uniform_(self.conv1.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.conv2.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
            nn.init.kaiming_uniform_(self.fc2.weight, mode='fan_in', nonlinearity='relu')
        elif weight_init == 'random':
            nn.init.uniform_(self.conv1.weight)
            nn.init.uniform_(self.conv2.weight)
            nn.init.uniform_(self.fc1.weight)
            nn.init.uniform_(self.fc2.weight)
        
        self.activation_func = activation_func

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation_func(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.activation_func(x)
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 16 * 16)
        x = self.fc1(x)
        x = self.activation_func(x)
        x = self.fc2(x)
        return x

activation_functions = {
    'relu': nn.ReLU(),
    'tanh': nn.Tanh(),
    'leaky_relu': nn.LeakyReLU(0.1)
}

In [5]:
# Step 5: Define the Training Function
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10, config_name=None):
    model = model.to(device)
    best_accuracy = 0.0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_accuracy = 100 * correct / total
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Acc: {val_accuracy}')

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model_state = model.state_dict()

    if best_model_state is not None:
        best_model_path = f'/kaggle/working/{config_name}_best.pth'
        torch.save(best_model_state, best_model_path)
        print(f'Best model for {config_name} saved at {best_model_path} with accuracy: {best_accuracy}')

    return best_accuracy

In [6]:
# Step 6: Redirect Output and Train the Model
class Tee:
    def __init__(self, *files):
        self.files = files

    def write(self, obj):
        for f in self.files:
            f.write(obj)

    def flush(self):
        for f in self.files:
            f.flush()

# Save the original sys.stdout
original_stdout = sys.stdout

# Open the output file
output_file = open('/kaggle/working/training_log.txt', 'w')

# Redirect stdout to both the file and the notebook
sys.stdout = Tee(sys.stdout, output_file)

best_accuracy_all = 0.0
best_config_all = None
best_model_all = None

criterion = nn.CrossEntropyLoss()

for activation_name, activation_func in activation_functions.items():
    for weight_init in ['xavier', 'kaiming', 'random']:
        for optimizer_name in ['SGD', 'Adam', 'RMSprop']:
            config_name = f'{activation_name}_{weight_init}_{optimizer_name}'
            print(f'\nTraining with {config_name}')
            model = CNN(activation_func, weight_init)
            if optimizer_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
            elif optimizer_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=0.001)
            elif optimizer_name == 'RMSprop':
                optimizer = optim.RMSprop(model.parameters(), lr=0.001)
            
            best_accuracy_config = train_model(model, train_loader, val_loader, optimizer, criterion, config_name=config_name)
            print(f'Best accuracy for {config_name}: {best_accuracy_config}')

            if best_accuracy_config > best_accuracy_all:
                best_accuracy_all = best_accuracy_config
                best_config_all = config_name
                best_model_all = model

best_model_path = '/kaggle/working/best_model_all.pth'
torch.save(best_model_all.state_dict(), best_model_path)
print(f'\nOverall best model saved at {best_model_path} with configuration: {best_config_all}, Accuracy: {best_accuracy_all}')

# Close the output file and restore the original sys.stdout
output_file.close()
sys.stdout = original_stdout


Training with relu_xavier_SGD


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 0.6447872076279078, Val Loss: 0.5927675473384368, Val Acc: 68.91025641025641
Epoch 2, Loss: 0.594045048340773, Val Loss: 0.5741923833504702, Val Acc: 70.33253205128206
Epoch 3, Loss: 0.5732796935316844, Val Loss: 0.5537706430141742, Val Acc: 72.27564102564102
Epoch 4, Loss: 0.5503492743158952, Val Loss: 0.5348416452224438, Val Acc: 73.75801282051282
Epoch 5, Loss: 0.533324675873304, Val Loss: 0.5278224509495956, Val Acc: 73.67788461538461
Epoch 6, Loss: 0.5180993397266437, Val Loss: 0.5115773685467548, Val Acc: 75.22035256410257
Epoch 7, Loss: 0.5007007920589203, Val Loss: 0.4942919955803798, Val Acc: 76.88301282051282
Epoch 8, Loss: 0.4854540853546216, Val Loss: 0.4850869889442737, Val Acc: 77.76442307692308
Epoch 9, Loss: 0.46854141507393277, Val Loss: 0.47566173397577727, Val Acc: 77.80448717948718
Epoch 10, Loss: 0.44934068811245453, Val Loss: 0.4749568372200697, Val Acc: 77.62419871794872
Best model for relu_xavier_SGD saved at /kaggle/working/relu_xavier_SGD_best.p

In [7]:
# Step 7: Display the log file in the notebook
with open('/kaggle/working/training_log.txt', 'r') as file:
    log_content = file.read()
    print(log_content)


Training with relu_xavier_SGD
Epoch 1, Loss: 0.6447872076279078, Val Loss: 0.5927675473384368, Val Acc: 68.91025641025641
Epoch 2, Loss: 0.594045048340773, Val Loss: 0.5741923833504702, Val Acc: 70.33253205128206
Epoch 3, Loss: 0.5732796935316844, Val Loss: 0.5537706430141742, Val Acc: 72.27564102564102
Epoch 4, Loss: 0.5503492743158952, Val Loss: 0.5348416452224438, Val Acc: 73.75801282051282
Epoch 5, Loss: 0.533324675873304, Val Loss: 0.5278224509495956, Val Acc: 73.67788461538461
Epoch 6, Loss: 0.5180993397266437, Val Loss: 0.5115773685467548, Val Acc: 75.22035256410257
Epoch 7, Loss: 0.5007007920589203, Val Loss: 0.4942919955803798, Val Acc: 76.88301282051282
Epoch 8, Loss: 0.4854540853546216, Val Loss: 0.4850869889442737, Val Acc: 77.76442307692308
Epoch 9, Loss: 0.46854141507393277, Val Loss: 0.47566173397577727, Val Acc: 77.80448717948718
Epoch 10, Loss: 0.44934068811245453, Val Loss: 0.4749568372200697, Val Acc: 77.62419871794872
Best model for relu_xavier_SGD saved at /kaggle